<a href="https://colab.research.google.com/github/AbdElRahmanElGharib/Pedestrian-Detection/blob/main/Pedestrian_Detection_(alpha).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install -q keras-cv
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d adityasinghz/pedestrian-and-car-dataset
!unzip -q pedestrian-and-car-dataset.zip
!cp -R /content/data/trainval/images /content/images
!cp /content/data/trainval/annotations/bbox-annotations.json /content/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.9/601.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 51.6 MB/s eta 0:00:00


Saving kaggle.json to kaggle.json
100% 699M/702M [00:42<00:00, 18.6MB/s]
100% 702M/702M [00:42<00:00, 17.5MB/s]


In [2]:
import cv2
import json
import pandas as pd
import tensorflow as tf
import keras_cv
import numpy as np

Using TensorFlow backend


In [3]:
print("Available Physical Devices: ", tf.config.list_physical_devices())

Available Physical Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
IM_PATH = 'image_000000001.jpg'

json_file = open('./bbox-annotations.json')
annotations = json.load(json_file)

im_id = -1

for image in annotations['images']:
    if image['file_name'] == IM_PATH:
        im_id = image['id']

objects = []

for annotation in annotations['annotations']:
    if annotation['image_id'] == im_id:
        objects.append(annotation)

print(objects)

[{'category_id': 1, 'image_id': 0, 'segmentation': [], 'iscrowd': 0, 'bbox': [846, 145, 146, 477], 'area': 0.08855438232421875, 'id': 0, 'license': 2}, {'category_id': 1, 'image_id': 0, 'segmentation': [], 'iscrowd': 0, 'bbox': [848, 216, 175, 551], 'area': 0.12261072794596355, 'id': 1, 'license': 2}, {'category_id': 2, 'image_id': 0, 'segmentation': [], 'iscrowd': 0, 'bbox': [74, 159, 75, 81], 'area': 0.007724761962890625, 'id': 2, 'license': 2}, {'category_id': 2, 'image_id': 0, 'segmentation': [], 'iscrowd': 0, 'bbox': [153, 124, 658, 643], 'area': 0.5379918416341146, 'id': 3, 'license': 2}]


In [5]:
im = cv2.imread('./images/'+IM_PATH)

im_bb = im
for object in objects:
    if object['category_id'] == 1:
        _class = 'Person'
    elif object['category_id'] == 2:
        _class = 'Car'
    else:
        _class = 'Other'

    x,y,w,h = object['bbox'][0], object['bbox'][1], object['bbox'][2], object['bbox'][3]
    im_bb = cv2.rectangle(im_bb, (x,y), (x+w, y+h), (0,255,0), 2)
    im_bb = cv2.putText(im_bb, _class, (x, y-7), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 255), 2)

cv2.imwrite('./im.png', im_bb)

True

In [6]:
data = pd.DataFrame(columns=['image_id', 'image_file_name', 'image_height', 'image_width', 'object_class', 'x', 'y', 'w', 'h'])
data

,image_id,image_file_name,image_height,image_width,object_class,x,y,w,h


In [7]:
images_file_names_list = []
for image in annotations['images']:
    images_file_names_list.append((image['file_name'], image['height'], image['width']))

In [8]:
for annotation in annotations['annotations']:
    if annotation['category_id'] == 1:
        _class = 'Person'
    elif annotation['category_id'] == 2:
        _class = 'Car'
    else:
        _class = 'Other'

    x,y,w,h = annotation['bbox'][0], annotation['bbox'][1], annotation['bbox'][2], annotation['bbox'][3]
    data.loc[len(data['image_id'])] = [annotation['image_id'],
                                       images_file_names_list[annotation['image_id']][0],
                                       images_file_names_list[annotation['image_id']][1],
                                       images_file_names_list[annotation['image_id']][2],
                                       _class, x, y, w, h]

In [9]:
data.head()

,image_id,image_file_name,image_height,image_width,object_class,x,y,w,h
0,0,image_000000001.jpg,768,1024,Person,846,145,146,477
1,0,image_000000001.jpg,768,1024,Person,848,216,175,551
2,0,image_000000001.jpg,768,1024,Car,74,159,75,81
3,0,image_000000001.jpg,768,1024,Car,153,124,658,643
4,1,image_000000002.jpg,576,1024,Person,488,281,28,71


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16772 entries, 0 to 16771
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   image_id         16772 non-null  int64 
 1   image_file_name  16772 non-null  object
 2   image_height     16772 non-null  int64 
 3   image_width      16772 non-null  int64 
 4   object_class     16772 non-null  object
 5   x                16772 non-null  int64 
 6   y                16772 non-null  int64 
 7   w                16772 non-null  int64 
 8   h                16772 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 1.3+ MB


In [11]:
data.describe()

,image_id,image_height,image_width,x,y,w,h
count,16772.000000,16772.000000,16772.000000,16772.000000,16772.000000,16772.000000,16772.000000
mean,1090.303363,742.922371,999.082220,422.889339,270.059027,146.697353,165.467326
std,642.705912,125.908542,102.926163,300.326034,174.522806,205.134140,168.542397
min,0.000000,282.000000,575.000000,0.000000,0.000000,0.000000,3.000000
25%,523.000000,683.000000,1024.000000,152.750000,133.000000,35.000000,56.000000
50%,1082.000000,696.000000,1024.000000,407.500000,261.000000,70.000000,108.000000
75%,1630.000000,768.000000,1024.000000,669.000000,376.000000,154.000000,208.000000
max,2238.000000,3000.000000,4496.000000,3883.000000,998.000000,3903.000000,2999.000000


In [12]:
data = data.convert_dtypes()

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16772 entries, 0 to 16771
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   image_id         16772 non-null  Int64 
 1   image_file_name  16772 non-null  string
 2   image_height     16772 non-null  Int64 
 3   image_width      16772 non-null  Int64 
 4   object_class     16772 non-null  string
 5   x                16772 non-null  Int64 
 6   y                16772 non-null  Int64 
 7   w                16772 non-null  Int64 
 8   h                16772 non-null  Int64 
dtypes: Int64(7), string(2)
memory usage: 1.4 MB


In [14]:
IM_SIZE = 320

m = 10000

X = []
Y = {'boxes': [],
     'classes': []}

for i, image in enumerate(images_file_names_list):
    if i >= m:
        break

    im = cv2.imread('./images/' + image[0])
    # im = cv2.resize(im, (int(data['image_width'].mean()), int(data['image_height'].mean())))
    im = cv2.resize(im, (IM_SIZE, IM_SIZE))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    X.append(im)

grouped = data.groupby('image_id')

gen_boxes = []
gen_classes = []

for i, group in enumerate(grouped.groups):
    if i >= m:
        break

    group_df = grouped.get_group(group)

    im_boxes = []
    im_classes = []
    im_num_detections = len(group_df['x'])

    for _, row in group_df.iterrows():

        x_min = row['x']/row['image_width']
        y_min = row['y']/row['image_height']
        x_max = (row['x']+row['w'])/row['image_width']
        y_max = (row['y']+row['h'])/row['image_height']

        box = [x_min, y_min, x_max, y_max]

        if row['object_class'] == 'Person':
            _class = 1.
        else:
            _class = 0.

        im_boxes.append(box)
        im_classes.append(_class)

    for i in range(100-im_num_detections):

        im_boxes.append(np.random.uniform(0., 1., (4,)))
        im_classes.append(0.)

    gen_boxes.append(im_boxes)
    gen_classes.append(im_classes)

Y['boxes'] = np.array(gen_boxes)
Y['classes'] = np.array(gen_classes)
X = np.array(X)

print(X.shape)
print(Y['boxes'].shape)
print(Y['classes'].shape)

(2239, 320, 320, 3)
(2239, 100, 4)
(2239, 100)


In [15]:
with tf.device('/GPU:0'):
  model = keras_cv.models.YOLOV8Detector(backbone=keras_cv.models.YOLOV8Backbone.from_preset(preset='yolo_v8_xl_backbone_coco', load_weights=True),
                                       num_classes=2, bounding_box_format='rel_xyxy')

124353200/124353200 [==============================] - 7s 0us/step


In [16]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, global_clipnorm=10),
              classification_loss='binary_crossentropy', box_loss='ciou')

In [17]:
for i, l in enumerate(model.layers):
    if i < 152:
        l.trainable = False

In [18]:
model.summary(expand_nested=True)

Model: "yolov8_detector"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 model (Functional)          {'P3': (None, None, None,    3097208   ['input_2[0][0]']             
                             320),                        0                                       
                              'P4': (None, None, None,                                            
                             640),                                                                
                              'P5': (None, None, None,                                            
                             640)}                                                  

In [19]:
model.fit(x=X, y=Y, batch_size=8, epochs=3)

Epoch 1/3
280/280 [==============================] - 234s 554ms/step - loss: 5.2628 - box_loss: 3.3007 - class_loss: 1.9621
Epoch 2/3
280/280 [==============================] - 144s 514ms/step - loss: 3.9922 - box_loss: 2.8816 - class_loss: 1.1106
Epoch 3/3
280/280 [==============================] - 142s 509ms/step - loss: 3.9499 - box_loss: 2.8468 - class_loss: 1.1032


In [20]:
im = cv2.imread('./images/' + data['image_file_name'][10000])
im = cv2.resize(im, (IM_SIZE, IM_SIZE))
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

In [21]:
input = tf.constant(im, name='input')
input = tf.expand_dims(input=input, axis=0)

In [22]:
output = model.predict(input)

1/1 [==============================] - 10s 10s/step


In [29]:
output['num_detections']

array([1], dtype=int32)

In [23]:
box = output['boxes'][0][0]
confidence = output['confidence'][0][0]

print(box)

box = box * np.array([data['image_width'][10000], data['image_height'][10000],
                      data['image_width'][10000], data['image_height'][10000]])

box = box.astype(int)

[0.0396637  0.09631786 0.89077055 0.93338454]


In [24]:
print('Predicted Bounding Box:\t', box, '\tConfidence:\t', confidence)

Predicted Bounding Box:	 [ 40  65 912 636] 	Confidence:	 0.28528434


In [25]:
im = cv2.imread('./images/'+data['image_file_name'][10000])

x_min, y_min, x_max, y_max = box[0], box[1], box[2], box[3]
im = cv2.rectangle(im, (x_min,y_min), (x_max, y_max), (0,255,0), 2)

cv2.imwrite('./im.png', im)

True

In [44]:
im = cv2.imread('./images/' + data['image_file_name'][0])
im = cv2.resize(im, (IM_SIZE, IM_SIZE))
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

input = tf.constant(im, name='input')
input = tf.expand_dims(input=input, axis=0)

output = model.predict(input)

1/1 [==============================] - 0s 81ms/step


In [45]:
output['num_detections']

array([1], dtype=int32)

In [46]:
box = output['boxes'][0][0]
confidence = output['confidence'][0][0]

print(box)

box = box * np.array([data['image_width'][0], data['image_height'][0],
                      data['image_width'][0], data['image_height'][0]])

box = box.astype(int)

print('Predicted Bounding Box:\t', box, '\tConfidence:\t', confidence)

[0.03968792 0.09624281 0.890889   0.93340224]
Predicted Bounding Box:	 [ 40  73 912 716] 	Confidence:	 0.28520024


In [47]:
im = cv2.imread('./images/'+data['image_file_name'][0])

x_min, y_min, x_max, y_max = box[0], box[1], box[2], box[3]
im = cv2.rectangle(im, (x_min,y_min), (x_max, y_max), (0,255,0), 2)

cv2.imwrite('./im2.png', im)

True